In [ ]:
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
%matplotlib inline

# import seaborn as sns; sns.set()

In [ ]:
import logging

LOG_LEVEL = "INFO"
LOG_FORMAT = '%(asctime)-15s %(levelname)-9s %(name)s: %(message)s'
logging.basicConfig(format=LOG_FORMAT, level=getattr(logging, LOG_LEVEL))

log = logging.getLogger("rbm")

# Restricted Boltzmann Machines (RBMs)

In [ ]:
v_0 = X_train[0]
h_0 = rbm.sample_hidden(v_0)

In [ ]:
v_1 = v_0 + 1

In [ ]:
v = np.repeat(v_0.reshape(1, -1), 3, axis=0)
h = np.repeat(h_0.reshape(1, -1), 3, axis=0)

v[0] = v[0] + 1
v[1] = v[1] + 2

In [ ]:
v.shape, h.shape

In [ ]:
np.matmul(v, rbm.c) + np.matmul(h, rbm.b)

In [ ]:
np.matmul(h, rbm.W.T).shape

In [ ]:
x = np.matmul(h, rbm.W.T)
x[0, :].shape

In [ ]:
np.matmul(h, np.matmul(v, rbm.W).T)

In [ ]:
np.diag(np.matmul(np.matmul(h, rbm.W.T), v.T))

In [ ]:
(np.dot(rbm.c, v_0) + np.dot(rbm.b, h_0))

In [ ]:
np.dot(v_0, np.dot(rbm.W, h_0))

In [ ]:
(np.dot(rbm.c, v_1) + np.dot(rbm.b, h_0))

In [ ]:
np.dot(v_1, np.dot(rbm.W, h_0))

In [ ]:
(np.matmul(v, rbm.c) +
 np.matmul(h, rbm.b) +
 np.diag(np.matmul(np.matmul(h, rbm.W.T), v.T)))

In [ ]:
rbm2 = BernoulliRBM(784, 500)

In [ ]:
rbm.b.shape, np.matmul(v, rbm2.W)

In [ ]:
rbm2.proba_hidden(v).shape == h.shape

In [ ]:
rbm2.proba_visible(h).shape == v.shape

In [ ]:
rbm2.sample_hidden(v).shape == h.shape

In [ ]:
rbm2.sample_visible(h).shape == v.shape

In [ ]:
v__0, v_k = rbm2.contrastive_divergence(v, k=4)
dc, db, dw = rbm2.grad(v__0, v_k)

In [ ]:
dc.shape, db.shape, dw.shape

In [ ]:
rbm2.step(v)

In [ ]:
X_train[:5].shape

In [ ]:
[rbm.free_energy(x) for x in X_train[:5]]

In [ ]:
rbm2.free_energy(X_train[:5])

In [ ]:
rbm.c = rbm2.c
rbm.b = rbm2.b
rbm.W = rbm2.W

In [ ]:
for i in range(len(X_train[:5])):
    a = [rbm.proba_hidden(x) for x in X_train[:5]][i]
    b = rbm2.proba_hidden(X_train[:5])[i]
    eps = 0.000001

    assert np.all((b + eps > a) & (b - eps < a))

In [ ]:
# gradients
v = X_train[:5]
v_k = rbm2.sample_visible(rbm2.sample_hidden(X_train[:5]))

a = rbm2.grad(v, v_k)
b = [rbm.grad(v[i], v_k[i]) for i in range(len(v))]

In [ ]:
b = [np.array(x) for x in list(zip(*b))]

In [ ]:
eps = 0.00001

In [ ]:
[np.all(np.all((b[i] + eps > a[i]) & (b[i] - eps < a[i]))) for i in range(len(a))]

In [ ]:
rbm2.free_energy(X_train[:5])

In [ ]:
np.array([rbm.free_energy(x) for x in X_train[:5]])

In [ ]:
visible = np.matmul(v, rbm2.c)
hidden = rbm2.b + np.matmul(v, rbm2.W)
- (visible + np.sum(np.log(1 + np.exp(hidden)), axis=1))

In [ ]:
    class BatchBernoulliRBM(object):
        """
        RBM with Bernoulli variables for hidden and visible states.
        """
        def __init__(self, num_visible, num_hidden):
            super(BatchBernoulliRBM, self).__init__()
            self.num_visible = num_visible
            self.num_hidden = num_hidden

            self.c, self.b, self.W = self.initialize(self.num_visible, self.num_hidden)

        @staticmethod
        def initialize(num_visible, num_hidden):
            # biases for visible and hidden, respectively
            c = np.zeros(num_visible)
            b = np.zeros(num_hidden)

            # weight matrix
            W = np.random.normal(0.0, 0.01, (num_visible, num_hidden))

            return c, b, W

        def energy(self, v, h):
            return - (np.matmul(v, self.c) +
                      np.matmul(h, self.b) +
                      np.matmul(h, np.matmul(v, rbm.W).T))

        def proba_visible(self, h):
            "Computes p(v | h)."
            return sigmoid(self.c + np.matmul(h, self.W.T))

        def proba_hidden(self, v):
            "Computes p(h | h)."
            return sigmoid(self.b + np.matmul(v, self.W))

        def sample_visible(self, h):
            "Samples visible units from the given hidden units `h`."
            # compute p(V_j = 1 | h)
            probas = self.proba_visible(h)
            # equiv. of V_j ~ p(V_j | h)
            rands = np.random.random(size=probas.shape)
            v = (probas > rands).astype(int)
            return v

        def sample_hidden(self, v):
            "Samples hidden units from the given visible units `v`."
            # compute p(H_{\mu} = 1 | v)
            probas = self.proba_hidden(v)
            # euqiv. of H_{\mu} ~ p(H_{\mu} | h)
            rands = np.random.random(size=probas.shape)
            h = (probas > rands).astype(np.int)
            return h

        def free_energy(self, v):
            # unnormalized
            # F(v) = - log \tilde{p}(v) = - \log \sum_{h} \exp ( - E(v, h))
            # using Eq. 2.20 (Fischer, 2015) for \tilde{p}(v)
            if len(v.shape) < 2:
                v = v.reshape(1, -1)
            visible = np.matmul(v, self.c)
            hidden = self.b + np.matmul(v, self.W)
            return - (visible + np.sum(np.log(1 + np.exp(hidden)), axis=1))

        def contrastive_divergence(self, v_0, k=1):
            v = v_0
            for t in range(k):
                h = self.sample_hidden(v)
                v = self.sample_visible(h)

            return v_0, v

        def grad(self, v_0, v_k):
            "Estimates the gradient of the negative log-likelihood using CD-k."
            proba_h_0 = self.proba_hidden(v_0)
            proba_h_k = self.proba_hidden(v_k)

            delta_c = v_0 - v_k
            delta_b = proba_h_0 - proba_h_k

            x = proba_h_0.reshape(proba_h_0.shape[0], 1, proba_h_0.shape[1])
            y = v_0.reshape(v_0.shape[0], v_0.shape[1], 1)
            z_0 = np.matmul(y, x)

            x = proba_h_k.reshape(proba_h_k.shape[0], 1, proba_h_k.shape[1])
            y = v_k.reshape(v_k.shape[0], v_k.shape[1], 1)
            z_k = np.matmul(y, x)
            delta_W = z_0 - z_k

            return delta_c, delta_b, delta_W

        def step(self, vs, k=1, lr=0.1, lmda=0.0):
            "Performs a single gradient ascent step using CD-k on the batch `vs`."
            v_0, v_k = self.contrastive_divergence(v, k=k)

            # compute gradient for each observed visible configuration
            dc, db, dW = self.grad(v_0, v_k)

            # accumulate gradients
            delta_c = np.mean(dc, axis=0)
            delta_b = np.mean(db, axis=0)
            delta_W = np.mean(dW, axis=0)

            # update parameters
            self.c += lr * delta_c
            self.b += lr * delta_b
            self.W += lr * delta_W

            # possible apply weight-decay
            if lmda > 0.0:
                self.c -= lmda * self.c
                self.b -= lmda * self.b
                self.W -= lmda * self.W

        def fit(self, train_data, k=1, learning_rate=0.01, num_epochs=5, batch_size=64, test_data=None):
            num_samples = train_data.shape[0]
            indices = np.arange(num_samples)
            np.random.shuffle(indices)

            loglikelihood_train = []
            loglikelihood = []

            for epoch in range(1, num_epochs + 1):
                # compute train & test negative log-likelihood
                nll_train = np.mean(rbm.free_energy(train_data))
                # nll_train = np.mean(np.array([rbm.free_energy(v) for v in train_data]))
                loglikelihood_train.append(nll_train)
                log.info(f"[{epoch} / {NUM_EPOCHS}] NLL (train): {nll_train:>20.5f}")

                if test_data is not None:
                    nll = np.mean(np.array([rbm.free_energy(v) for v in test_data]))
                    log.info(f"[{epoch} / {NUM_EPOCHS}] NLL (test):  {nll:>20.5f}")
                    loglikelihood.append(nll)

                # iterate through dataset in batches
                bar = tqdm(total=num_samples)
                for start in range(0, num_samples, batch_size):
                    # ensure we don't go out-of-bounds
                    end = min(start + batch_size, num_samples)

                    # take a gradient-step
                    rbm.step(train_data[start: end], k=k, lr=learning_rate)

                    # update progress
                    bar.update(end - start)

                bar.close()

                # shuffle indices for next epoch
                np.random.shuffle(indices)

            # compute train & test negative log-likelihood of final batch
            nll_train = np.mean([rbm.free_energy(v) for v in train_data])
            loglikelihood_train.append(nll_train)
            log.info(f"[{epoch} / {NUM_EPOCHS}] NLL (train): {nll_train:>20.5f}")

            if test_data is not None:
                nll = np.mean([rbm.free_energy(v) for v in test_data])
                log.info(f"[{epoch} / {NUM_EPOCHS}] NLL (test):  {nll:>20.5f}")
                loglikelihood.append(nll)

            return loglikelihood_train, loglikelihood

        def loss(self, samples_true, per_sample_hidden=100):
            """
            Computes the difference in empirical distributions observed in `samples_true` and samples
            of visible units obtained from the model, using the same initial state.

            Parameters
            ----------
            samples_true: array-like
                True samples of visible units to compare to.
            per_sample_hidden: int
                Number of `h` to "partially marginalize" over when computing p(v) = p(v | h) p(h).

            Returns
            -------
            loss: float
                Difference between the two distributions.

            Notes
            -----
            Can be very inaccurate estimate of how well the model is performing since one is NOT completely
            marginalizing out all hidden variables.

            """
            k = per_sample_hidden
            # the loss is the log energy-difference between the p(v) and p(v_k), where `v_k` is the Gibbs sampled visible unit
            return np.mean([
                self.free_energy(v) - self.free_energy(self.contrastive_divergence(v, k)[1])
                for v in samples_true
            ])

        def reconstruct(self, v, num_samples=100):
            samples = self.sample_visible(self.sample_hidden(v))
            for _ in range(num_samples - 1):
                samples += self.sample_visible(self.sample_hidden(v))

            probs = samples / num_samples 

            return probs


In [ ]:
def sigmoid(z):
    # clip the values due to possibility of overflow
    return 1.0 / (1.0 + np.exp(-np.maximum(np.minimum(z, 30), -30)))


### Restricted Boltzmann Machine ###
class BernoulliRBM(object):
    """
    RBM with Bernoulli variables for hidden and visible states.
    """
    def __init__(self, num_visible, num_hidden):
        super(BernoulliRBM, self).__init__()
        self.num_visible = num_visible
        self.num_hidden = num_hidden

        self.c, self.b, self.W = self.initialize(self.num_visible, self.num_hidden)
      
    @staticmethod
    def initialize(num_visible, num_hidden):
        # biases for visible and hidden, respectively
        c = np.zeros(num_visible)
        b = np.zeros(num_hidden)

        # weight matrix
        # Results on MNIST are highly dependent on this initialization
        W = np.random.normal(0.0, 0.01, (num_visible, num_hidden))
        
        return c, b, W

    def energy(self, v, h):
        return - (np.dot(self.c, v) + np.dot(self.b, h) + np.dot(v, np.dot(self.W, h)))

    def proba_visible(self, h):
        "Computes p(v | h)."
        return sigmoid(self.c + np.dot(self.W, h))

    def proba_hidden(self, v):
        "Computes p(h | h)."
        return sigmoid(self.b + np.dot(self.W.T, v))

    def sample_visible(self, h):
        "Samples visible units from the given hidden units `h`."
        # compute p(V_j = 1 | h)
        probas = self.proba_visible(h)
        # equiv. of V_j ~ p(V_j | h)
        rands = np.random.random(size=probas.shape)
        v = (probas > rands).astype(int)
        return v

    def sample_hidden(self, v):
        "Samples hidden units from the given visible units `v`."
        # compute p(H_{\mu} = 1 | v)
        probas = self.proba_hidden(v)
        # euqiv. of H_{\mu} ~ p(H_{\mu} | h)
        rands = np.random.random(size=probas.shape)
        h = (probas > rands).astype(np.int)
        return h

    def free_energy(self, v):
        # unnormalized
        # F(v) = - log \tilde{p}(v) = - \log \sum_{h} \exp ( - E(v, h))
        # using Eq. 2.20 (Fischer, 2015) for \tilde{p}(v)
        visible = np.dot(self.c, v)
        hidden = self.b + np.dot(self.W.T, v)
        return - (visible + np.sum(np.log(1 + np.exp(hidden))))
    
    def contrastive_divergence(self, v_0, k=1):
        h = self.sample_hidden(v_0)
        v = self.sample_visible(h)
        
        if k > 1:
            for t in range(k):
                h = self.sample_hidden(v)
                v = self.sample_visible(h)
                
        return v_0, v

    def grad(self, v_0, v_k):
        "Estimates the gradient of the negative log-likelihood using CD-k."
        proba_h_0 = self.proba_hidden(v_0)
        proba_h_k = self.proba_hidden(v_k)
        
        delta_c = v_0 - v_k
        delta_b = proba_h_0 - proba_h_k

        # reshape so that we can compute v_j h_{\mu} by
        # taking the dot product to obtain `delta_W`
        v_0 = np.reshape(v_0, (-1, 1))
        proba_h_0 = np.reshape(proba_h_0, (1, -1))
        
        v_k = np.reshape(v_k, (-1, 1))
        proba_h_k = np.reshape(proba_h_k, (1, -1))
        
        delta_W = np.dot(v_0, proba_h_0) - np.dot(v_k, proba_h_k)
        
        return delta_c, delta_b, delta_W
    
    def step(self, vs, k=1, lr=0.1, lmda=0.0):
        "Performs a single gradient ascent step using CD-k on the batch `vs`."
        # TODO: can we perform this over the batch using matrix multiplication instead?
        v_0, v_k = self.contrastive_divergence(vs[0], k=k)
        delta_c, delta_b, delta_W = self.grad(v_0, v_k)
        for v in vs[1:]:
            # perform CD-k
            v_0, v_k = self.contrastive_divergence(v, k=k)
            # compute gradient for each observed visible configuration
            dc, db, dW = self.grad(v_0, v_k)
            # accumulate gradients
            delta_c += dc
            delta_b += db
            delta_W += dW

        # update parameters
        self.c += lr * (delta_c / len(vs))
        self.b += lr * (delta_b / len(vs))
        self.W += lr * (delta_W / len(vs))

        # possible apply weight-decay
        if lmda > 0.0:
            self.c -= lmda * self.c
            self.b -= lmda * self.b
            self.W -= lmda * self.W
        
    def loss(self, samples_true, per_sample_hidden=100):
        """
        Computes the difference in free energy, i.e.
        
            log p(v_k) - log p(v_0)
            
        where `v_0` is sample from data, and `v_k` is the sample from the chain initialized at v_0.

        Parameters
        ----------
        samples_true: array-like
            True samples of visible units to compare to.
        per_sample_hidden: int
            Number of `h` to "partially marginalize" over when computing p(v) = p(v | h) p(h).

        Returns
        -------
        loss: float
            Difference between the two distributions.
            
        Notes
        -----
        Could also consider the reconstruction error, i.e. how much the mean-squared error between the
        observed v_0 and the generated v_k

        """
        # the loss is the log energy-difference between the p(v) and p(v_k), where `v_k` is the Gibbs sampled visible unit
        return np.mean([
            self.free_energy(v) - self.free_energy(self.contrastive_divergence(v, per_sample_hidden)[1])
            for v in samples_true
        ])

    def reconstruct(self, v, num_samples=100):
        samples = self.sample_visible(self.sample_hidden(v))
        for _ in range(num_samples - 1):
            samples += self.sample_visible(self.sample_hidden(v))

        probs = samples / num_samples 

        return probs


### Quickly making sure everything works

In [ ]:
# testah
num_visible = 100
num_hidden = 50
rbm = BernoulliRBM(num_visible, num_hidden)
# v = np.random.randint(2, size=num_visible)
v = np.random.randint(2, size=num_visible)
h = np.random.randint(2, size=num_hidden)
rbm.energy(v, h)

In [ ]:
rbm.W.shape

In [ ]:
rbm.b.shape

In [ ]:
h = rbm.sample_hidden(v)
v = rbm.sample_visible(h)
print(v)

In [ ]:
v_0, v_k = rbm.contrastive_divergence(v, k=10)
v_0, v_k

In [ ]:
dc, db, dW = rbm.grad(v_0, v_k)

In [ ]:
dc.shape, db.shape, dW.shape

In [ ]:
rbm.W.max(), rbm.c.max(), rbm.b.max()

# MNIST

## Load data

In [ ]:
import os
from six.moves import urllib
from sklearn.datasets import fetch_mldata

# Alternative method to load MNIST, since mldata.org is often down...
from scipy.io import loadmat
mnist_alternative_url = "https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat"
mnist_path = "./mnist-original.mat"

if os.path.exists(mnist_path):
    log.info(f"Found existing file at {mnist_path}; loading...")
    mnist_raw = loadmat(mnist_path)
    mnist = {
        "data": mnist_raw["data"].T,
        "target": mnist_raw["label"][0],
        "COL_NAMES": ["label", "data"],
        "DESCR": "mldata.org dataset: mnist-original",
    }
else:
    log.info(f"Dataset not found at {mnist_path}; downloading...")
    response = urllib.request.urlopen(mnist_alternative_url)
    with open(mnist_path, "wb") as f:
        content = response.read()
        f.write(content)
    mnist_raw = loadmat(mnist_path)
    mnist = {
        "data": mnist_raw["data"].T,
        "target": mnist_raw["label"][0],
        "COL_NAMES": ["label", "data"],
        "DESCR": "mldata.org dataset: mnist-original",
    }
    log.info("Success!")

## Split into training and test

In [ ]:
# train-test split
from sklearn.model_selection import train_test_split

# in case we want to use `cupy` to run on the GPU
X = np.asarray(mnist["data"])
y = np.asarray(mnist["target"])

X_train, X_test, y_train, y_test = train_test_split(X, y)

# clip values since we're working with binary variables and original images have domain [0, 255]
X_train = X_train.clip(0, 1)
X_test = X_test.clip(0, 1)

## Train

In [ ]:
from tqdm import tqdm

In [ ]:
# model parameters
USE_GPU = False

if USE_GPU:
    import cupy as np
else:
    import numpy as np

NUM_HIDDEN = 500

# model
rbm = BernoulliRBM(X_train.shape[1], NUM_HIDDEN)

In [ ]:
rbm.W.shape, rbm.c.shape, rbm.b.shape

In [ ]:
# training parameters
NUM_EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.1
K = 1

# train
log.info(f"Starting training")
num_samples = X_train.shape[0]
indices = np.arange(num_samples)
np.random.shuffle(indices)

loglikelihood = []
loglikelihood_train = []

for epoch in range(1, NUM_EPOCHS + 1):
    # compute train & test negative log-likelihood
    nll_train = np.mean([rbm.free_energy(v) for v in X_train])
    loglikelihood_train.append(nll_train)
    log.info(f"[{epoch} / {NUM_EPOCHS}] NLL (train): {nll_train:>20.5f}")
    
    nll = np.mean([rbm.free_energy(v) for v in X_test])
    log.info(f"[{epoch} / {NUM_EPOCHS}] NLL (test):  {nll:>20.5f}")
    loglikelihood.append(nll)

    # iterate through dataset in batches
    bar = tqdm(total=num_samples)
    for start in range(0, num_samples, BATCH_SIZE):
        # ensure we don't go out-of-bounds
        end = min(start + BATCH_SIZE, num_samples)
        
        # take a gradient-step
        rbm.step(X_train[start: end], k=K, lr=LEARNING_RATE)
        
        # update progress
        bar.update(end - start)
        
    bar.close()

    # shuffle indices for next epoch
    np.random.shuffle(indices)

In [ ]:
plt.plot(- np.array(loglikelihood))

## Display reconstructions of test-samples

In [ ]:
def flip(x, p):
    x = x.copy()
    mask = np.random.random(size=x.shape) < p
    flipped = (~(x.astype(np.bool))).astype(np.int)
    x[mask] = flipped[mask]
    return x

In [ ]:
# display a couple of testing examples
import matplotlib.pyplot as plt

# plot some reconstructions
from matplotlib import gridspec

n_rows = 6
n_cols = 8

fig, axes = plt.subplots(n_rows, n_cols, 
                         sharex=True, sharey=True, 
                         figsize=(16, 12),
                         # make it tight
                         gridspec_kw=dict(wspace=-0.1, hspace=-0.01))

for i in range(n_rows):
    for j in range(n_cols // 2):
        v_0 = X_test[np.random.randint(X_test.shape[0])]
        
        # introduce noise
        noise = 0.1
        v = flip(v_0, noise)
        
        probs = rbm.reconstruct(v, num_samples=1000)
#         num_samples = 100
#         k = 100
#         _, probs = rbm.contrastive_divergence(v, k=k)

#         for _ in range(num_samples - 1):
#             probs += rbm.contrastive_divergence(v, k=k)[1]
        
#         probs = probs / num_samples

        # in case we've substituted with `cupy`
        if np.__name__ != "numpy":
            v = np.asnumpy(v)
            probs = np.asnumpy(probs)
            
        log.info(f"Reconstruction error of {(i, 2 * j)}: {np.mean(np.abs(v_0 - probs))} (noise: {noise})")

        axes[i][2 * j].imshow(np.reshape(v, (28, 28)))
        axes[i][2 * j + 1].imshow(np.reshape(probs, (28, 28)))

        # customization; remove labels
        axes[i][2 * j].set_xticklabels([])
        axes[i][2 * j].set_yticklabels([])

        axes[i][2 * j + 1].set_xticklabels([])
        axes[i][2 * j + 1].set_yticklabels([])

# Ising Model

In [ ]:
def create_updown_couplings(N, coupling_values=[0, 1]):
    "Returns an of dim N x N x N x N, representing 2D neighbor-couplings for each entry."
    J = np.zeros((N, N, N, N))
    
    for i in range(N):
        for j in range(N):
            if i + 1 < N:
                J[i, j, i + 1, j] = np.random.choice(coupling_values)
            if i - 1 > 0:
                J[i, j, i - 1, j] = np.random.choice(coupling_values)
            if j + 1 < N:
                J[i, j, i, j + 1] = np.random.choice(coupling_values)
            if j - 1 > 0:
                J[i, j, i, j - 1] = np.random.choice(coupling_values)
    return J

In [ ]:
import itertools as it

In [ ]:
import numpy as np


class Ising(object):
    """
    Ising model.
    """
#     boltzmann_constant = 1.38064852e-23
    
    def __init__(self, T, N, field=None, couplings=None, domain=[0, 1]):
        super(Ising, self).__init__()
        self.domain = domain
        self.N = N
#         self.beta = 1.0 / (self.boltzmann_constant * T)
        self.beta = 1.0 / T
        self.T = T
        
        self.h = field if field is not None else np.random.randn(N, N)
        self.J = couplings if couplings is not None else create_updown_couplings(N, self.domain)

    def energy(self, x):
        # (J[s] * x) gives us J_{s, i} * x_i for all i, and then we multiply by 
        couplings_energy = np.sum([
            np.sum(self.J[i, j] * x * x[i, j])
            for i in range(self.N)
            for j in range(self.N)])
        field_energy = np.sum(self.h * x)
        return - (field_energy + 0.5 * couplings_energy)
    

    def sample(self, state, full=True):
        """
        Parameters
        ----------
        n: int
            Number of timesteps to sample for

        """
        if full:
            indices = it.product(range(N), range(N))
        else:
            indices = [np.random.randint(self.N, size=2)]
        
        for (i, j) in indices:
            delta_E = np.sum(self.J[i, j] * state)
    #         delta_E = 2 * (delta_E + self.h[i, j])
            delta_E = (delta_E + self.h[i, j])

            u = np.random.uniform()
            new_state = state.copy()

            if delta_E > 0:
                if np.exp(- self.beta * delta_E) > u:
                    # accept flip
                    new_state[i, j] = self.domain[-1] if state[i, j] < self.domain[-1] else self.domain[0]
            else:
                # flip
                new_state[i, j] = self.domain[-1] if state[i, j] < self.domain[-1] else self.domain[0]
                
            # update
            state = new_state

        return new_state

In [ ]:
# testing
N = 10

domain = [0, 1]
couplings = create_updown_couplings(N, coupling_values=domain)
couplings[couplings != 0.0] = 1.0

# state = np.random.choice(domain, size=(N, N))
state = np.zeros((N, N)) + 1.0
state[N // 2, N // 2] = domain[0]
state[N // 2, N // 2 + 1] = domain[0]

ising = Ising(33, N, field=np.zeros((N, N)), couplings=couplings)
energy = ising.energy(state)
energy

In [ ]:
x = state
x

In [ ]:
history = []

In [ ]:
# burn-in
for _ in range(N * N):
    x = ising.sample(x)
    
for _ in range(10000):
    x = ising.sample(x)
    history.append(x)

In [ ]:
import time
from IPython import display

fig = plt.figure(figsize=(8, 8))

# burn-in
for _ in range(N * N):
    x = ising.sample(x)

while True:
    plt.imshow(x)
    for _ in range(N):
        x = ising.sample(x)
        history.append(x)
#     x = ising.sample(x)
    display.clear_output(wait=True)
    display.display(plt.gcf())
    time.sleep(0.5)

In [ ]:
len(history)

## Training a RBM on an Ising Model

In [ ]:
history = np.array([np.reshape(x, -1) for x in history])

In [ ]:
X_train, X_test = train_test_split(history)

In [ ]:
# parameters
USE_GPU = False

if USE_GPU:
    import cupy as np
else:
    import numpy as np

NUM_HIDDEN = 500

In [ ]:
rbm = BernoulliRBM(num_visible, NUM_HIDDEN)

In [ ]:
# training parameters
NUM_EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 0.1
K = 1

# train
log.info(f"Starting training")
num_samples = X_train.shape[0]
indices = np.arange(num_samples)
np.random.shuffle(indices)

loglikelihood = []
loglikelihood_train = []

for epoch in range(1, NUM_EPOCHS + 1):
    # # Might be a good idea to increase the `k` as we get nearer to the end, so that our gradient estimate
    # # is more accurate
    # if epoch == NUM_EPOCHS:
    #     log.info(f"[{epoch} / {NUM_EPOCHS}] Increasing k: {K} -> {5 * K}")
    #     K = 5 * K
    # else:
    nll_train = np.mean([rbm.free_energy(v) for v in X_train])
    loglikelihood_train.append(nll_train)
    log.info(f"[{epoch} / {NUM_EPOCHS}] NLL (train): {nll_train:>20.5f}")
    
    nll = np.mean([rbm.free_energy(v) for v in X_test])
    log.info(f"[{epoch} / {NUM_EPOCHS}] NLL (test):  {nll:>20.5f}")
    loglikelihood.append(nll)

    bar = tqdm(total=num_samples)
    for start in range(0, num_samples, BATCH_SIZE):
        # ensure we don't go out-of-bounds
        end = min(start + BATCH_SIZE, num_samples)
        
        # take a gradient-step
        rbm.step(X_train[start: end], k=K, lr=LEARNING_RATE)
        
        # update progress
        bar.update(end - start)
    bar.close()

    # shuffle indices for next epoch
    np.random.shuffle(indices)

In [ ]:
np.abs(rbm.b).max(), np.abs(rbm.c).max(), np.abs(rbm.W).max()

### Generating from Ising

In [ ]:
import time
from IPython import display

fig, axes = plt.subplots(1, 2, figsize=(16, 16))

v = X_test[np.random.randint(len(X_test))]
v_k = rbm.reconstruct(np.reshape(v, -1))

while True:
    
    axes[0].imshow(np.reshape(v, (N, N)))
    axes[1].imshow(np.reshape(v_k, (N, N)))

    v = ising.sample(np.reshape(v, (N, N)))
    
    # Reconstruct from `v` if you don't want the entire chain to be sampled
    # by the RBM, but only the next step
#     v_k = rbm.sample_visible(rbm.sample_hidden(np.reshape(v, -1)))
    v_k = rbm.reconstruct(np.reshape(v, -1))
    
    display.clear_output(wait=True)
    display.display(plt.gcf())
    time.sleep(0.5)